# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [1]:
#open the file
import pandas as pd
try:
    data = pd.read_csv('credit_scoring_eng.csv')
except:
    data = pd.read_csv('/datasets/credit_scoring_eng.csv')
#lemmatization, will be used later
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer()
#look at the general information
data.info()
data.describe()
data.head()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


### Conclusion

12 columns, 21525 rows, missing values in 2 columns - 10% of rows missing days_employed and 10% of rows missing total_income - both are quantitative values. Dtype float64 for both days_employed and total_income instead of int64. 54 duplicate rows. Some days_employed values are negative (which is impossible), I can't ask the owner for clarification in this case, and the column isn't very relevant to project goal. I'll use the absolute numeric value later. Will do the same for children column.

## Data preprocessing

### Processing missing values

In [2]:
#use absolute values for days_employed & children columns
data['days_employed'] = data['days_employed'].abs()
data['children'] = data['children'].abs()
#calculate percentage of missing data
data.isnull().sum()
data.isnull().sum() / len(data)
null_data = data[data.isnull().any(1)]
#check if missing data related to income_type
null_data.groupby('income_type')['income_type'].count()
#fill missing data with median values based on the same education, income_type and gender
for i in null_data.index:
    temp = []
    median_help_table = data[(data['education'] == data.iloc[i]['education']) & (data['income_type'] == data.iloc[i]['income_type']) & (data['gender'] == data.iloc[i]['gender'])].copy()
    temp.append(median_help_table['days_employed'].median())
    temp.append(median_help_table['total_income'].median())
    data.loc[i,'days_employed'] = temp[0]
    data.loc[i,'total_income'] = temp[1]
#check if missing data still exists
data.isnull().sum()
#just 1 missing data row left, safe to remove as it won't affect the results
data = data.dropna()
#check if missing data still exists
data.isnull().sum()

/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Conclusion

%10 of rows are missing value of both days_employed and total_income. Both are quantitative type.  It doesn't seem like there is any relation between the missing data, possibly a problem with the data input process. We can fill those missing values with representative values - median for the matching education, income_type and gender. After this process, there's just 1 row left with missing values, we are safe to drop it.

### Data type replacement

In [3]:
#change data type from float64 to int64
data['days_employed'] = data['days_employed'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')
#check the changes
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Conclusion

Data type for both days_employed and total_income changed from float64 to int64, using astype, to make calculation easier in later stages.

### Processing duplicates

In [4]:
#analyze which columns have duplicates
data['education'].unique()
data['family_status'].unique()
data['gender'].unique()
data['income_type'].unique()
data['purpose'].unique()
#fixing education column
data['education'] = data['education'].str.lower()
#check the change
data['education'].unique()
#column purpose needs fix, using lemmatization to fix
same_meaning_list=['house' ,'housing', 'estate', 'property', 'education', 'educated', 'university', 'wedding', 'car']
for i in data.index:
    words = nltk.word_tokenize(data['purpose'].loc[i])
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    for word in same_meaning_list:
        word_to_check = wordnet_lemma.lemmatize(word,pos='n')
        if word_to_check in lemmas:
            if (word == 'housing') or (word == 'estate') or (word == 'property') or (word == 'house'):
                data.loc[i,'purposes'] = 'property'
            elif (word == 'university') or (word == 'educated'):
                data.loc[i,'purposes'] = 'education'
            else:                
                data.loc[i,'purposes'] = word
#check the change
data.isnull().sum()
data['purposes'].unique()
#drop purpose as it's irrelevant now that we have purposes column
data = data.drop(['purpose'], axis = 1)
data['purposes'].value_counts()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purposes
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,property
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,property
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,wedding


### Conclusion

We have 2 fields with duplicates values: education - case sensitivity duplicates, purpose - values with different wording but same meaning, needs Lemmatization to fix. Possible reasons for the duplicate data: education - automatic uppercase was enabled for part of the customers while filling the data, purpose - customers filled this value with their own words instead of choosing from a list of options. We can add a new column to fix the different wording for the same purpose. I'll remove purpose column as it's not needed anymore.

### Categorizing Data

In [5]:
data['total_income'].describe()
#function to return the correct income class based on income
def income_class(value):
    if value < 15000:
        return 'below average income'
    elif value >= 15000 and value < 50000:
        return 'average income'
    elif value >= 50000 and value < 100000:
        return 'above average income'
    else:
        return 'high income'
#apply the function to total_income column, create income_class column
data['income_class'] = data['total_income'].apply(income_class)
data.groupby(['income_class','debt'])['debt'].agg('count')
data['income_class'].value_counts()
#apply function to children and create has_children column
data['children'].value_counts()
data['has_children'] = data['children'].apply(lambda x: 'No' if x == 0 else 'Yes')
data['has_children'].value_counts()
#drop both columns, irrelevant now that we have categorized data 
data = data.drop(['total_income'], axis = 1)
data = data.drop(['children'], axis = 1)
data.head()

,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,purposes,income_class,has_children
0,8437,42,bachelor's degree,0,married,0,F,employee,0,property,average income,Yes
1,4024,36,secondary education,1,married,0,F,employee,0,car,average income,Yes
2,5623,33,secondary education,1,married,0,M,employee,0,property,average income,No
3,4124,32,secondary education,1,married,0,M,employee,0,education,average income,Yes
4,340266,53,secondary education,1,civil partnership,1,F,retiree,0,wedding,average income,No


### Conclusion

We have 2 columns that could use categorizing to help us analyze the data: total_income and children.
After looking at the data, I decided to group total_income to 4 groups that can represent the income according to the amount of customers. Children values have some abnormalities like 70+ customers with 20 children. We can pass that information to the data owner. The number of customers without children at all is much larger than with 1/2/3/4/5 children, so I decided to group the data to: Yes for has children, No for no children.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [6]:
children = pd.pivot_table(data, index = 'has_children', columns = 'debt', 
                               values = 'family_status_id', aggfunc = 'count', margins = True).reset_index()

children['default_rate'] = (children[1]/children['All']) * 100

children

debt,has_children,0,1,All,default_rate
0,No,13085,1063,14148,7.513429
1,Yes,6698,678,7376,9.191974
2,All,19783,1741,21524,8.088645


### Conclusion

The default rate for customers with children is higher, 22% increase compared to customers without children. Loan to customers with children is riskier.

- Is there a relation between marital status and repaying a loan on time?

In [7]:
family_status = pd.pivot_table(data, index = 'family_status', columns = 'debt', 
                               values = 'family_status_id', aggfunc = 'count', margins = True).reset_index()

family_status['default_rate'] = (family_status[1]/family_status['All']) * 100

family_status

debt,family_status,0,1,All,default_rate
0,civil partnership,3789,388,4177,9.288963
1,divorced,1110,85,1195,7.112971
2,married,11448,931,12379,7.520801
3,unmarried,2539,274,2813,9.740491
4,widow / widower,897,63,960,6.562500
5,All,19783,1741,21524,8.088645


### Conclusion

Civil partnership and unmarried customers default rate is significantly higher than the average so loan to those customers is more riskier, while widow/widower customers rate is significantly lower than the average, so loan is more safe for the bank for them.

- Is there a relation between income level and repaying a loan on time?

In [8]:
income = pd.pivot_table(data, index = 'income_class', columns = 'debt', 
                               values = 'family_status_id', aggfunc = 'count', margins = True).reset_index()

income['default_rate'] = (income[1]/income['All']) * 100

income

debt,income_class,0,1,All,default_rate
0,above average income,1136,86,1222,7.037643
1,average income,15105,1351,16456,8.209772
2,below average income,3449,298,3747,7.953029
3,high income,93,6,99,6.060606
4,All,19783,1741,21524,8.088645


### Conclusion

It is less riskier to loan to customers with above average income. The default rate for above average income customers is 13% less than the average, and 25% less for high income customers.

- How do different loan purposes affect on-time repayment of the loan?

In [9]:
purpose = pd.pivot_table(data, index = 'purposes', columns = 'debt', 
                               values = 'family_status_id', aggfunc = 'count', margins = True).reset_index()

purpose['default_rate'] = (purpose[1]/purpose['All']) * 100

purpose

debt,purposes,0,1,All,default_rate
0,car,3912,403,4315,9.339513
1,education,3652,370,4022,9.199403
2,property,10057,782,10839,7.214688
3,wedding,2162,186,2348,7.921635
4,All,19783,1741,21524,8.088645


### Conclusion

Car and education purposes default rate is significantly higher than the average so loan to those customers is more riskier.

## General conclusion

The genral conclusion is that there is definitely a relation between customers marital status and number of children to whether they will default on a loan. 

## Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.